In [4]:
# importing required libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import os
## plase change to folder path
working_dir = "/home/zhuoyan/nlp/Induction-Head"
# working_dir = "/Users/zyxu/Documents/py/NLP/induction_head/induc_head"
os.chdir(working_dir)
import json 
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
from src.config import Config
from src.model.tinyGPT2 import tinyGPT2LMHeadModel
from src.data.gen_simple import gen_simple_data
from src.utils import fix_random_seed, create_folder
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


from transformers import GPT2Config


In [9]:
tinymodel = tinyGPT2LMHeadModel.from_pretrained("model_output")

In [10]:
tinymodel

tinyGPT2LMHeadModel(
  (transformer): tinyGPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x tinyGPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
d_model = 768
model = tinymodel.transformer
W_q = model.h[1].mha.W_q.weight.numpy(force=True)
W_k = model.h[1].mha.W_k.weight.numpy(force=True)
W_v = model.h[0].mha.W_v.weight.numpy(force=True)
W_o = model.h[0].mha.W_o.weight.numpy(force=True)
W_qk = W_q.T @ W_k / np.sqrt(d_model)
W_ov = W_o @ W_v

AttributeError: 'tinyGPT2Block' object has no attribute 'mha'

In [ ]:
sns.heatmap(W_qk)
plt.show()
sns.heatmap(W_ov)
plt.show()

b_q = model.h[1].mha.W_q.bias.numpy(force=True)
b_k = model.h[1].mha.W_k.bias.numpy(force=True)
beta_q = (W_q.T @ b_k.reshape(-1,1)).ravel()  / np.sqrt(d_model)
beta_k = (W_k @ b_q.reshape(-1,1)).ravel() / np.sqrt(d_model)

vals = np.zeros(len_pattern)
for j in range(len_pattern):
    vals[j] = attn2.squeeze()[len_pattern+j,j+1]
plt.plot(vals) # check whether induction head appears, and yes!
plt.show()